In [1]:
import numpy as np
import pandas as pd
import faiss
import pickle
from sklearn.preprocessing import MinMaxScaler, RobustScaler, Normalizer, StandardScaler
import json
import tqdm

In [2]:
minmaxscaler = StandardScaler()

In [3]:
drop_columns = ["6", "44", "70", "21", "25", "33", "59", "65"]

In [4]:
data_base = pd.read_csv("../Masterskaya-2/data/base.csv", index_col=0)
data_validation = pd.read_csv("../Masterskaya-2/data/validation.csv", index_col=0)
data_validation_answer = pd.read_csv("../Masterskaya-2/data/validation_answer.csv", index_col=0)

In [ ]:
# data_base = pd.read_csv("../Masterskaya-2/data/base.csv", index_col=0)
data_validation = pd.read_csv("../Masterskaya-2/data/validation.csv", index_col=0)
data_validation_answer = pd.read_csv("../Masterskaya-2/data/validation_answer.csv", index_col=0)["Expected"]
# data_train = pd.read_csv("./data/train.csv", index_col=0)

data_base = data_base.drop(drop_columns, axis=1)
data_validation = data_validation.drop(drop_columns, axis=1)
# data_train = data_train.drop(drop_columns, axis=1)
# data_train_target = data_train["Target"]
# data_train = data_train.drop(["Target"], axis=1)

In [ ]:
base_index = {k: v for k, v in enumerate(pd.DataFrame(data_base).index.to_list())}

In [ ]:
data_base = minmaxscaler.fit_transform(data_base)
data_validation = minmaxscaler.transform(data_validation)
# data_train = minmaxscaler.transform(data_train)

In [ ]:
minmaxscaler = None

In [ ]:
# with open('../Masterskaya-2/models/model.pkl', 'rb') as fid:
#     model = pickle.load(fid)
#     fid.close()

In [ ]:
# data_validation = pd.DataFrame(model.predict(data_validation))

In [ ]:
model = faiss.read_index("../Masterskaya-2/models/model.faiss")
model = faiss.index_cpu_to_all_gpus(model)
# model.nprobe = 5

In [ ]:
vecs, idx = model.search(np.ascontiguousarray(data_validation).astype('float32'), 10)

In [ ]:
model = None

In [ ]:
# acc = 0
# for target, el in zip(data_validation_answer.values.tolist(), idx.tolist()):
#     acc += int(target in [base_index[r] for r in el])
# print(100 * acc / len(idx))

In [ ]:
targets = {}
for target, el in tqdm.tqdm(zip(data_validation_answer.values.tolist(), idx.tolist())):
    # try:
    #     targs = [base_index[r] for r in el]
    #     targs.remove(target)
    # except:
    #     targs = [base_index[r] for r in el]
    targs = [base_index[r] for r in el]
    targets[data_validation_answer[data_validation_answer == target].index.to_list()[0]] = targs
    

100000it [11:02, 150.99it/s]


In [ ]:
with open("./data/predictions_targets.json", "w") as file:
    json.dump(targets, file, )
    